In [ ]:
from PIL import Image
from PIL.ExifTags import TAGS, GPSTAGS
import os, glob, sys
import ogr, osr, math as m, datetime


class ImageMetaData(object):
    '''
    Extract the exif data from any image. Data includes GPS coordinates, 
    Focal Length, Manufacture, and more.
    '''
    exif_data = None
    image = None

    def __init__(self, img_path):
        self.image = Image.open(img_path)
        #print(self.image._getexif())
        self.get_exif_data()
        super(ImageMetaData, self).__init__()

    def get_exif_data(self):
        """Returns a dictionary from the exif data of an PIL Image item. Also converts the GPS Tags"""
        exif_data = {}
        info = self.image._getexif()


        if info:
            for tag, value in info.items():
                decoded = TAGS.get(tag, tag)
                if decoded == "GPSInfo":
                    gps_data = {}
                    for t in value:
                        sub_decoded = GPSTAGS.get(t, t)
                        gps_data[sub_decoded] = value[t]

                    exif_data[decoded] = gps_data
                else:
                    exif_data[decoded] = value
        self.exif_data = exif_data
        return exif_data

    def get_if_exist(self, data, key):
        if key in data:
            return data[key]
        return None

    def convert_to_degress(self, value):

        """Helper function to convert the GPS coordinates 
        stored in the EXIF to degress in float format"""
        d0 = value[0][0]
        d1 = value[0][1]
        d = float(d0) / float(d1)

        m0 = value[1][0]
        m1 = value[1][1]
        m = float(m0) / float(m1)

        s0 = value[2][0]
        s1 = value[2][1]
        s = float(s0) / float(s1)

        return d + (m / 60.0) + (s / 3600.0)

    def get_lat_lng(self):
        """Returns the latitude and longitude, if available, from the provided exif_data (obtained through get_exif_data above)"""
        lat = None
        lng = None
        gps_imgdirection = None
        exif_data = self.get_exif_data()

        
        if "GPSInfo" in exif_data:      
            gps_info = exif_data["GPSInfo"]
            gps_latitude = self.get_if_exist(gps_info, "GPSLatitude")
            gps_latitude_ref = self.get_if_exist(gps_info, 'GPSLatitudeRef')
            gps_longitude = self.get_if_exist(gps_info, 'GPSLongitude')
            gps_longitude_ref = self.get_if_exist(gps_info, 'GPSLongitudeRef')
            
            #gps_imgdirection = _get_if_exist(gps_info, 'GPSImgDirection')

            if gps_latitude and gps_latitude_ref and gps_longitude and gps_longitude_ref:
                lat = self.convert_to_degress(gps_latitude)
                if gps_latitude_ref != "N":                     
                    lat = 0 - lat
                lng = self.convert_to_degress(gps_longitude)
                if gps_longitude_ref != "E":
                    lng = 0 - lng
        return lat, lng  #, gps_imgdirection

def createEsriShapefile(exportfilename):
    import osr
    import ogr

    dstProjection = osr.SpatialReference()
    dstProjection.ImportFromEPSG(4326)
    #Transformationsparameter
    driver = ogr.GetDriverByName('ESRI Shapefile')
    if os.path.exists(exportfilename):
        driver.DeleteDataSource(exportfilename)
    #Zieldatensatz vorbereiten:
    
    print(50*"^")
    print (exportfilename)
    destinationFile = driver.CreateDataSource(exportfilename)
    lyrname=exportfilename.split(".")[0]
    destinationLayer = destinationFile.CreateLayer(lyrname,dstProjection, geom_type=ogr.wkbPolygon)    

    #Festlegung der Attribute
    fieldDef = ogr.FieldDefn('Id', ogr.OFTInteger)
    destinationLayer.CreateField(fieldDef)
    fieldDef = ogr.FieldDefn("Filename",ogr.OFTString)
    fieldDef.SetWidth(200)
    destinationLayer.CreateField(fieldDef)
    fieldDef = ogr.FieldDefn("Yaw", ogr.OFTReal)
    destinationLayer.CreateField(fieldDef)
    fieldDef = ogr.FieldDefn("Pitch", ogr.OFTReal)
    destinationLayer.CreateField(fieldDef)
    fieldDef = ogr.FieldDefn("Roll", ogr.OFTReal)
    destinationLayer.CreateField(fieldDef)
    fieldDef = ogr.FieldDefn("lngcenter", ogr.OFTReal)
    destinationLayer.CreateField(fieldDef)
    fieldDef = ogr.FieldDefn("latcenter", ogr.OFTReal)
    destinationLayer.CreateField(fieldDef)
    fieldDef = ogr.FieldDefn("imagewidth", ogr.OFTReal)
    destinationLayer.CreateField(fieldDef)
    fieldDef = ogr.FieldDefn("imagheight", ogr.OFTReal)
    destinationLayer.CreateField(fieldDef)
    fieldDef = ogr.FieldDefn("igroundw", ogr.OFTReal)
    destinationLayer.CreateField(fieldDef)
    fieldDef = ogr.FieldDefn("igroundh", ogr.OFTReal)
    destinationLayer.CreateField(fieldDef)
    
    destinationFile = None

    return destinationFile
    
def getfl(path,fltype, exporttextfile,expimgfp):
    expfl = open(exporttextfile, "w")

    #create Esri Shapefile
    exportfilename = path +"/"+ str(expimgfp) #"photoFrames.shp"
    print(exportfilename)
    exportfile = createEsriShapefile(exportfilename)
    expshpfl = ogr.Open(exportfilename,1)
    shpLayer = expshpfl.GetLayer(0)
            

    filetype = "*."+fltype
    for infile in glob.glob( os.path.join(path, filetype) ):
            infile=str(infile)
            infile = infile.replace("\\","/")
            print("Bearbeite ", infile)
            
            meta_data =  ImageMetaData(infile)
            latlng = list(meta_data.get_lat_lng())

            #exif_data = meta_data.get_exif_data()
            #print(exif_data)
            lat = latlng[0]
            lng = latlng[1]
            xp,yp = reprojectPoint(4326,21781,lng,lat)
            print("  Cameralocation: ", xp," ",yp)

            #pitch,roll,yaw,breite,laenge = getXMPinfo(infile)
            pitch,roll,yaw,cen2bot,cen2top,cen2lef,cen2rig = getXMPinfo(infile)
            
            
            y1=yp+cen2bot
            x1=xp+cen2lef
            y2=yp+cen2top
            x2=xp+cen2lef
            y3=yp+cen2top
            x3=xp+cen2rig
            y4=yp+cen2bot
            x4=xp+cen2rig


            print(100*"/")
            print(100*"/")
            print("Before Rotation:")
            print("x1,y1,x2,y2,x3,y3,x4,y4: ",x1,y1,x2,y2,x3,y3,x4,y4)
            print(100*"/")
            print(100*"/")
            
            p1x,p1y,p2x,p2y,p3x,p3y,p4x,p4y = computeOrthogonals(xp,yp,float(yaw+180.0),cen2bot,cen2top,cen2lef,cen2rig)

            laenge = cen2rig - cen2lef
            breite = cen2top - cen2bot

            lngp1,latp1 = reprojectPoint(21781,4326,p1x,p1y)
            lngp2,latp2 = reprojectPoint(21781,4326,p2x,p2y)
            lngp3,latp3 = reprojectPoint(21781,4326,p3x,p3y)
            lngp4,latp4 = reprojectPoint(21781,4326,p4x,p4y)
            
            #Create SpatialFeature
            ftrName = infile
            linearRing = ogr.Geometry(ogr.wkbLinearRing)
            linearRing.AddPoint(lngp1,latp1)
            linearRing.AddPoint(lngp2,latp2)
            linearRing.AddPoint(lngp3,latp3)
            linearRing.AddPoint(lngp4,latp4)
            linearRing.AddPoint(lngp1,latp1)
            rect = ogr.Geometry(ogr.wkbPolygon)
            rect.AddGeometry(linearRing)
            rectfeature = ogr.Feature(shpLayer.GetLayerDefn())
            rectfeature.SetGeometry(rect)
            rectfeature.SetField("Filename", ftrName)
            rectfeature.SetField("Yaw", yaw)
            rectfeature.SetField("Pitch", pitch)
            rectfeature.SetField("Roll", roll)
            rectfeature.SetField("lngcenter", lng)
            rectfeature.SetField("latcenter", lat)
            #rectfeature.SetField("imagewidth", laenge)
            #rectfeature.SetField("imageheight", breite)
            rectfeature.SetField("igroundh", breite)
            rectfeature.SetField("igroundw", laenge)
            shpLayer.CreateFeature(rectfeature)
            rectfeature.Destroy()            
            
            print(lngp1,latp1,lngp2,latp2,lngp3,latp3,lngp4,latp4)
            
            expstring = "%s;%s;%s;%s;%s;%s;%s;%s\n" %(infile,lng,lat,xp,yp,pitch,roll,yaw)
            expfl.write(expstring)
    expfl.close()
    expshpfl = None

def reprojectPoint(sourceSRS,destSRS,x,y):
    source = osr.SpatialReference()
    source.ImportFromEPSG(sourceSRS)
    target = osr.SpatialReference()
    target.ImportFromEPSG(destSRS)
    transform = osr.CoordinateTransformation(source, target)
    point = ogr.CreateGeometryFromWkt("POINT (%f %f)" %(x,y))
    point.Transform(transform)

    pnt = str(point).replace("POINT (","").replace(")","")
    projpntx = pnt.split(" ")[0]   
    projpnty = pnt.split(" ")[1]
    
    return float(projpntx),float(projpnty)
    
    
def getValueFromKey(xmpString,keyword):
    val = xmpString.split(keyword+"=")[1].split("\\n")[0].replace("\"","")    
    
    return val

def getXMPinfo(filename):
    with open( filename, "rb") as fin:
        img = fin.read()
    imgAsString=str(img)
    xmp_start = imgAsString.find('<x:xmpmeta')
    xmp_end = imgAsString.find('</x:xmpmeta')
    if xmp_start != xmp_end:
        xmpString = imgAsString[xmp_start:xmp_end+12]

    #print(xmpString) 
    #print ()

    GimbalYaw = getValueFromKey(xmpString,"GimbalYawDegree")
    GimbalPitch = getValueFromKey(xmpString,"GimbalPitchDegree")
    GimbalRoll = getValueFromKey(xmpString,"GimbalRollDegree")
    FlightYaw = getValueFromKey(xmpString,"FlightYawDegree")
    FlightPitch = getValueFromKey(xmpString,"FlightPitchDegree")
    FlightRoll = getValueFromKey(xmpString,"FlightRollDegree")
    RelativeAltitude = getValueFromKey(xmpString,"RelativeAltitude")
    
    #Pitch,Roll,Yaw: https://www.pinterest.ch/pin/465700417698798293/
    #Sensor Dimensions:
    #6.17 mm x 3.47 mm (0.243 in x 0.136 in)
    #from: http://vfxcamdb.com/dji-mavic/ and http://metapicz.com/#makernotes-box
    #Weitere Def: https://developer.dji.com/mobile-sdk/documentation/introduction/flightController_concepts.html
    #http://www.agisoft.com/forum/index.php?topic=7623.0
    '''
    Definitionen:
    xsensor = laengere Seite der Kamera, entspricht Horizontalen, entspricht y-Achse Drohne; Roll ist relevant
    ysensor = kuerzere Seite der Kamera, entspricht Vertikalen, entspricht x-Achse Drohne; Pitch ist relevant
    '''
    xsensor = 6.17  #gem. Verhältnis Pixel: 4000x3000         #6.44228 #6.17
    ysensor = 4.627                                           #4.83171 #3.47 #4.627
    focallen = 4.73                                             #4.73
    xpixel = 4000
    ypixel = 3000

    altitude = float(RelativeAltitude)
    xgimbal = float(GimbalRoll)+float(FlightRoll)
    ygimbal = 90.0 + float(GimbalPitch)+float(FlightPitch)
    zgimbal = float(GimbalYaw)+float(FlightYaw)
    
    fieldofViewWide = 2*m.degrees(m.atan(xsensor/(2*focallen)))
    fieldofViewTall = 2*m.degrees(m.atan(ysensor/(2*focallen)))
    fromDroneToBottomOfPicture = altitude*m.tan(m.radians(ygimbal - 0.5*fieldofViewTall))
    fromDroneToTopOfPicture = altitude*m.tan(m.radians(ygimbal + 0.5*fieldofViewTall))
    fromDroneToLeftOfPicture = altitude*m.tan(m.radians(xgimbal - 0.5*fieldofViewWide))
    fromDroneToRightOfPicture = altitude*m.tan(m.radians(xgimbal + 0.5*fieldofViewWide))
    widthOfPhotoFootprint = fromDroneToRightOfPicture - fromDroneToLeftOfPicture
    heightOfPhotoFootprint = fromDroneToTopOfPicture - fromDroneToBottomOfPicture

    print("%"*200)
    print("fieldofViewWide, fieldofViewTall, fromDroneToBottomOfPicture, fromDroneToTopOfPicture, fromDroneToLeftOfPicture, fromDroneToRightOfPicture, laenge, breite:")
    print("   ",fieldofViewWide, " / ", fieldofViewTall," / ", fromDroneToBottomOfPicture," / ", fromDroneToTopOfPicture," / ", fromDroneToLeftOfPicture," / ", fromDroneToRightOfPicture, " / ", (fromDroneToRightOfPicture-fromDroneToLeftOfPicture), " / ", (fromDroneToTopOfPicture-fromDroneToBottomOfPicture))
    print("xgimbal, ygimbal, zgimbal: ", xgimbal, " / ", ygimbal, " / ", zgimbal)
    print("Altitude: ", RelativeAltitude)
    print("%"*200)
    
    
    return xgimbal,ygimbal,zgimbal,float(fromDroneToBottomOfPicture),float(fromDroneToTopOfPicture),float(fromDroneToLeftOfPicture),float(fromDroneToRightOfPicture)

def rotation(alpha,cenx,ceny,x1,y1,x2,y2,x3,y3,x4,y4):
    xr1,yr1 = getrotation(alpha,cenx,ceny,x1,y1)
    xr2,yr2 = getrotation(alpha,cenx,ceny,x2,y2)
    xr3,yr3 = getrotation(alpha,cenx,ceny,x4,y3)
    xr4,yr4 = getrotation(alpha,cenx,ceny,x4,y4)
    
    print(100*"/")
    print(100*"/")
    print("xr1,yr1,xr2,yr2,xr3,yr3,xr4,yr4: ",xr1,yr1,xr2,yr2,xr3,yr3,xr4,yr4)
    
    
    print(50*"*")
    print("Laenge: ",x2-x1, x3-x2, x4-x3, x1-x4)
    print("Breite: ",y2-y1, y3-y2, y4-y3, y1-y4)
    print("P0: %f/%f, P1: %f/%f, P2: %f/%f, P3: %f/%f" %(x1,y1,x2,y2,x3,y3,x4,y4))
    print("Berechnete Distanzen: ")
    print("  P0-P1: ",computeDistance(xr1,yr1,xr2,yr2))
    print("  P1-P2: ",computeDistance(xr2,yr2,xr3,yr3))
    print("  P2-P3: ",computeDistance(xr3,yr3,xr4,yr4))
    print("  P3-P0: ",computeDistance(xr4,yr4,xr1,yr1))
    print(50*"*")
    
    print(100*"/")
    print(100*"/")
    return xr1,yr1,xr2,yr2,xr3,yr3,xr4,yr4

def getrotation(alpha,cenx,ceny,x,y):              
    #orthogonal to polar:
    theta=m.degrees(m.atan(y-ceny)/(x-cenx))
    r = m.sqrt(m.pow((y-ceny),2)+m.pow((x-cenx),2))
    alphar=m.radians(theta+alpha)
    xr1 = cenx+r*m.cos(alphar)
    yr1 = ceny+r*m.sin(alphar)
    print("``````````````````````````````")
    print("``````````````````````````````")
    print("Theta,r,xr,yr: ",theta,r,xr1,yr1)
    print("``````````````````````````````")
    return xr1,yr1

def computeDistance(x1,y1,x2,y2):
    return m.sqrt(m.pow((x2-x1),2)+m.pow((y2-y1),2))

#def computeOrthogonals(mx,my,alpha_degrees,breite,laenge):
def computeOrthogonals(mx,my,alpha_degrees,cen2bot,cen2top,cen2lef,cen2rig):
    alpha = m.radians(alpha_degrees)
    
    ax=mx+cen2bot
    ay=my+cen2lef
    bx=mx+cen2top
    by=my+cen2lef
    cx=mx+cen2top
    cy=my+cen2rig
    dx=mx+cen2bot
    dy=my+cen2rig
    
    a1x=ax-mx
    a1y=ay-my
    b1x=bx-mx
    b1y=by-my
    c1x=cx-mx
    c1y=cy-my
    d1x=dx-mx
    d1y=dy-my

    a2x=a1x*m.cos(alpha)+a1y*m.sin(alpha)
    a2y=-a1x*m.sin(alpha)+a1y*m.cos(alpha)
    b2x=b1x*m.cos(alpha)+b1y*m.sin(alpha)
    b2y=-b1x*m.sin(alpha)+b1y*m.cos(alpha)
    c2x=c1x*m.cos(alpha)+c1y*m.sin(alpha)
    c2y=-c1x*m.sin(alpha)+c1y*m.cos(alpha)
    d2x=d1x*m.cos(alpha)+d1y*m.sin(alpha)
    d2y=-d1x*m.sin(alpha)+d1y*m.cos(alpha)

   
    a3x=mx+a2x
    a3y=my+a2y
    b3x=mx+b2x
    b3y=my+b2y
    c3x=mx+c2x
    c3y=my+c2y
    d3x=mx+d2x
    d3y=my+d2y
    
    laenge = cen2rig - cen2lef
    breite = cen2top - cen2bot
    print(50*"*")
    print("Laenge: ",laenge," Breite: ",breite," Alpha: ",alpha_degrees)
    print("P0: %f/%f, P1: %f/%f, P2: %f/%f, P3: %f/%f" %(ax,ay,bx,by,cx,cy,dx,dy))
    print("P0t: %f/%f, P1t: %f/%f, P2t: %f/%f, P3t: %f/%f" %(a3x,a3y,b3x,b3y,c3x,c3y,d3x,d3y))
    print("Berechnete Distanzen: ")
    print("  P0-P1: ",computeDistance(a3x,a3y,b3x,b3y))
    print("  P1-P2: ",computeDistance(b3x,b3y,c3x,c3y))
    print("  P2-P3: ",computeDistance(c3x,c3y,d3x,d3y))
    print("  P3-P0: ",computeDistance(d3x,d3y,a3x,a3y))
    print(50*"*")
    
    return a3x,a3y,b3x,b3y,c3x,c3y,d3x,d3y
    
################################################
# Example ######################################
################################################
if __name__ == "__main__":
    #image = ... # load an image through PIL's Image object
    curpath=input("Enter the path of drone-images: ") or "D:/SBB/Drohnendaten/Flug_Kraftwerkinsel_Birsfelden_20171103/block1"
    curfltpe=input("Enter Filetype (e.h. jpg)") or "jpg"
    
    exporttextfile = input("Enter filename for export cameraposition: ") or "image_exif_information.txt"
    
    ts = datetime.datetime.now().timestamp()
    defaultimgfpnm = "imgfootprints"+str(ts)+".shp"
    exportimgfootprints = input("Enter filename for imageframes: ") or defaultimgfpnm
    print("imgfootprints",ts,".shp")
    path = getfl(curpath, curfltpe, curpath+"/"+exporttextfile,exportimgfootprints)
    
    print()
    print()
    print(50*'*')
    print(50*'*')
    print(10*' ' + "F I N I S H E D")
    print(50*'*')
    print(50*'*')
